In [26]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 28, Finished, Available, Finished)

In [27]:
%%sql
create table if not exists Gold_Product
(
Product_ID LONG,
Product_Category string,
Product string,
Created_TS timestamp,
Modified_TS timestamp
)
using delta

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 29, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
Max_Date = spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Gold_Product").first()[0]

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 30, Finished, Available, Finished)

In [29]:
Max_Date

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 31, Finished, Available, Finished)

'1900-01-01'

In [30]:
df_bronze = spark.sql("""select distinct 
Product_Category,
Product
from Bronze_Sales where Modified_TS>'{}'
""".format(Max_Date))

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 32, Finished, Available, Finished)

In [31]:
Max_ID = spark.sql("select coalesce(max(Product_ID),0) from Gold_Product").first()[0]

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 33, Finished, Available, Finished)

In [32]:
df_final = df_bronze.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 34, Finished, Available, Finished)

In [33]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 35, Finished, Available, Finished)

In [34]:
%%sql
select * from ViewProduct limit 10

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 36, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 3 fields>

In [37]:
%%sql
merge into Gold_Product as GP 
USING ViewProduct as VP
on GP.Product=VP.Product and GP.Product_Category=VP.Product_Category
when matched then
update SET
gp.Modified_TS = CURRENT_TIMESTAMP()

when not matched THEN
INSERT
(
GP.Product_ID,
GP.Product_category,
gp.Product,
GP.Created_TS,
GP.Modified_TS
)
VALUES
(
VP.Product_ID,
VP.Product_category,
VP.Product,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP()
)

StatementMeta(, 45ee51bc-15c3-4dfa-a7a6-0bb2aeac3ec9, 39, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>